### BitsAndBytes
[BitsAndBytes](https://github.com/TimDettmers/bitsandbytes) 簡稱 BNB，是由 [Tim Dettmers](https://github.com/TimDettmers) 開發的專案，在 HF Transformers 套件裡面做 8-Bit 或 4-Bit 量化的後端技術之一。
* 8-Bit 的做法與上節所述大致相同，但是 BNB 更著重在**隔離異常值 (Isolate Outlier)** 上
    * 因為作者觀察到如果將一些數值過大或過小的異常值也量化掉，會造成很大的誤差，但如果將少量的異常值隔離出來，維持他們原本的精度，便能大幅減少誤差值。
* 屬於**訓練後量化 (Post-Training Quantization, PTQ)**。

#### 1. 首先安裝套件：

In [ ]:
%%bash
pip install bitsandbytes

#### 2. 透過 `BitsAndBytesConfig` 來將模型轉成 BNB 8-Bit 的格式：

In [ ]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM

bnb_config = BitsAndBytesConfig(load_in_8b=True)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    quantization_config=bnb_config
)

* 除了 8-Bit 以外，BNB 也有提供 4-Bit 格式。BNB 4-Bit 所使用的資料型態為 Normal Float 4 (NF4)，是 [QLoRA](https://arxiv.org/abs/2305.14314) 論文裡面提出的資料型態。
    * 要使用 BNB 4-Bit 的話，只需要把 `BitsAndBytesConfig` 裡的 `load_in_8bit` 改成 `load_in_4bit` 就可以了。
    * 除此之外還有一些其他參數可以設定：
        ```python
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )
        ```
    * 被量化的模型在實際進行矩陣運算時，是會被反向量化回浮點數的，預設是用 FP32 進行運算，也可以透過 `bnb_4bit_compute_dtype` 參數將其設定為 FP16 會算的比較快。
    * 預設 4-Bit 的資料型態是一般的 FP4，將 `bnb_4bit_quant_type` 設定為 NF4 就可以使用論文裡面提到的資料型態。
    * 最後如果開啟 `bnb_4bit_use_double_quant` 可以進一步減少模型的大小。
* 將模型壓縮到 4-Bit 不僅能讓我們順利在單張 24 GB 消費級顯卡上使用 30B 的模型進行推論，也可以結合 LoRA 訓練來微調 30B 的模型。